In [1]:
import pandas as pd
import numpy as np
import joblib as jb
from scipy.stats import spearmanr

In [2]:
train_train_level0 = jb.load("train_train_level0.pkl.z")
train_train_level1 = jb.load("train_train_level1.pkl.z")
train_valid_level1 = jb.load("train_valid_level1.pkl.z")
test = pd.read_feather("test.f")


X_train0, y_train0 = train_train_level0.filter(regex=r'feature', axis=1), train_train_level0['target_kazutsugi']
X_train1, y_train1 = train_train_level1.filter(regex=r'feature', axis=1), train_train_level1['target_kazutsugi']
X_val1, y_val1 = train_valid_level1.filter(regex=r'feature', axis=1), train_valid_level1['target_kazutsugi']
X_test, y_test = test.filter(regex=r'feature', axis=1), test['target_kazutsugi']

In [3]:
import glob
import re


preds_train1 = glob.glob("./preds_train1/*.pkl.z")
preds_val1 = glob.glob("./preds_val1/*.pkl.z")

df_train1 = []
for p_name in preds_train1:
    p = jb.load(p_name)
    p_name = re.search(r"train1/(.*)\.pkl.z", p_name).group(1)
    p_df = pd.DataFrame(p, columns=[p_name])
    df_train1.append(p_df)
    
    
df_val1 = [] 
scores_val1 = dict()
for p_name in preds_val1:
    p = jb.load(p_name)
    p_name = re.search(r"val1/(.*)\.pkl.z", p_name).group(1)
    p_df = pd.DataFrame(p, columns=[p_name])
    df_val1.append(p_df)
    scores_val1[p_name] = spearmanr(y_val1, p_df).correlation
    
    
df_train1 = pd.concat(df_train1, axis=1)
df_val1 = pd.concat(df_val1, axis=1)

/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/scipy/stats/stats.py:4196: SpearmanRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(SpearmanRConstantInputWarning())


In [4]:
df_train1.shape

(178773, 208)

In [5]:
df_train1.columns[~df_train1.columns.isin(df_val1.columns)]

Index([], dtype='object')

In [6]:
df_val1.shape

(180938, 208)

In [7]:
df_test = [] 
scores_test = dict()
preds_test = glob.glob("./preds_test/*.pkl.z")
for p_name in preds_test:
    p = jb.load(p_name)
    p_name = re.search(r"test/(.*)\.pkl.z", p_name).group(1)
    p_df = pd.DataFrame(p, columns=[p_name])
    df_test.append(p_df)
    scores_test[p_name] = spearmanr(y_test, p_df).correlation
    
    
df_test = pd.concat(df_test, axis=1)

# Selected cols (xgb + ridge)

In [8]:
from xgboost import XGBRegressor
selected_xgb = ['xgb_classifier', 'xgb_feature_groups_feature_charisma_feature_dexterity_feature_wisdom', 
            'xgb_feature_groups_feature_intelligence_feature_strength', 'lgbm_feature_groups_feature_constitution',
           'mlp_109_39_0.0009911332752617498', 'xgb_feature_groups_feature_intelligence_feature_charisma_feature_constitution',
           'lgbm_feature_groups_feature_strength_feature_dexterity', 'lgbm_feature_groups_feature_charisma_feature_dexterity_feature_constitution',
           'xgb_feature_groups_feature_intelligence_feature_wisdom']

Xtr = df_train1[selected_xgb].copy()
Xval = df_val1[selected_xgb].copy()
mdl = XGBRegressor(n_estimators=1000, learning_rate=0.01, max_depth=1, random_state=0, colsample_bytree=0.1, subsample=0.25, 
                   tree_method='gpu_hist', gpu_id=0)
mdl.fit(Xtr, y_train1)

p_xgb_s = mdl.predict(Xval)
c = spearmanr(y_val1, p_xgb_s).correlation
print(c)

0.047701264031541436


In [9]:
from sklearn.linear_model import Ridge
selected_ridge  = ['xgb_classifier', 'xgb_feature_groups_feature_intelligence_feature_charisma_feature_dexterity', 'xgb_depth_1', 'xgb_rhyper_5']

Xtr = df_train1[selected_ridge].copy()
Xval = df_val1[selected_ridge].copy()
mdl = Ridge(alpha=1.)
mdl.fit(Xtr, y_train1)

p_ridge_s = mdl.predict(Xval)
c = spearmanr(y_val1, p_ridge_s).correlation
print(c)

0.04617040351645618


In [14]:
p = 0.3*p_ridge_s + 0.7*p_xgb_s
c = spearmanr(y_val1, p).correlation
print(c)

0.04827477707807782


In [ ]:
# 0.04807860485862486
# 0.04822589672049141
# 0.04827477707807782


# Todas as cols (xgb + ridge)

In [15]:
mdl = XGBRegressor(n_estimators=1000, learning_rate=0.01, max_depth=1, random_state=0, colsample_bytree=0.1, subsample=0.25, 
                   tree_method='gpu_hist', gpu_id=0)
mdl.fit(df_train1, y_train1)
p_xgb_t = mdl.predict(df_val1)
c = spearmanr(y_val1, p_xgb_t).correlation
print(c)

0.044034288558525335


In [16]:
mdl = Ridge(alpha=1.)
mdl.fit(df_train1, y_train1)
p_ridge_t = mdl.predict(df_val1)
c = spearmanr(y_val1, p_ridge_t).correlation
print(c)

0.030028430650290143


In [21]:
p = 0.2*p_ridge_t + 0.8*p_xgb_t
c = spearmanr(y_val1, p).correlation
print(c)

0.04476200295534109


# TESTTT!!!

In [22]:
Xtr = df_train1[selected_xgb].copy()
mdl = XGBRegressor(n_estimators=1000, learning_rate=0.01, max_depth=1, random_state=0, colsample_bytree=0.1, subsample=0.25, 
                   tree_method='gpu_hist', gpu_id=0)
mdl.fit(Xtr, y_train1)

p_xgb_test = mdl.predict(df_test[selected_xgb])

In [23]:
from sklearn.linear_model import Ridge

Xtr = df_train1[selected_ridge].copy()
mdl = Ridge(alpha=1.)
mdl.fit(Xtr, y_train1)

p_ridge_test = mdl.predict(df_test[selected_ridge])

In [24]:
p = 0.3*p_ridge_test + 0.7*p_xgb_test
c = spearmanr(y_test, p).correlation
print(c)

0.025073960438983554


# best models 

In [28]:
best_val = 0
best_model = ''
for k,v in scores_val1.items():
    if v > best_val:
        print(k,v)
        best_val = v
        best_model = k
print("Test Score", scores_test[best_model])



lgbm_feature_groups_feature_intelligence_feature_charisma_feature_constitution 0.022858083685266956
xgb_rhyper_8 0.03968169496104963
xgb_boosted_rf_10 0.04199226728240279
xgb_boosted_rf_5 0.04235166048882618
xgb_default 0.04241387938098152
xgb_classifier 0.04290319376803716
Test Score 0.024535468740518818


In [32]:
print("Melhora val", 0.04827477707807782 / 0.04290319376803716 - 1 )
print("Melhor test", 0.025073960438983554 / 0.024535468740518818 - 1)

Melhora val 0.12520241124898446
Melhor test 0.021947479551326143


# E o feature set completo?

In [33]:
Xtr = df_train1
mdl = XGBRegressor(n_estimators=1000, learning_rate=0.01, max_depth=1, random_state=0, colsample_bytree=0.1, subsample=0.25, 
                   tree_method='gpu_hist', gpu_id=0)
mdl.fit(Xtr, y_train1)

p_xgb_test = mdl.predict(df_test)

In [34]:
Xtr = df_train1
mdl = Ridge(alpha=1.)
mdl.fit(Xtr, y_train1)

p_ridge_test = mdl.predict(df_test)

In [35]:
p = 0.2*p_ridge_test + 0.8*p_xgb_test
c = spearmanr(y_test, p).correlation
print(c)

0.024410911110673422


In [36]:
print("Melhora val", 0.04476200295534109 / 0.04290319376803716 - 1 )
print("Melhor test", 0.024410911110673422 / 0.024535468740518818 - 1)

Melhora val 0.04332566002787286
Melhor test -0.005076635427783582
